# Rendimiento escolar

Modelo que precide el promedio final del año escolar para un alumno, con fin de identificar bajo desempeño académico.

Se sugiere explorar atributos ambientales del alumno para ver si pueden ser abstraídos en categorías latentes.

### 1. Preliminares

A continuación se definen las características del problema.

#### 1.1 Problema

Variable objetivo: Promedio final del año escolar.

El problema pertenece a uno de regresión, dado que la variable objetivo, definida como el promedio final del año escolar, presenta valores continuos.

Enunciado: promedio final del año escolar para un individuo explicado por atributos personales y ambientales.

#### 1.2 Atributos disponibles

Para explicar el promedio final del año escolar de un estudiante, tenemos disponibles los siguientes atributos:

* school: categórica. Especifica la escuela.
* sex: categórica. Define el sexo.
* age: numérica. Detalla la edad.
* address: categórica. Especifica la ubicación del estudiante (urbana o rural).
* famsize: categórica. Define el tamaño de la familia (menor o igual a 3, mayor a 3).
* Pstatus: categórica. Detalla el estado cohabitacional de los padres.
* Medu: categórica (label encoding). Presenta el nivel educacional de la madre.
* Fedu: categórica (label encoding). Presenta el nivel educacional del padre.
* Mjob: categórica. Especifica la ocupación de la madre.
* Fjob: categórica. Define la ocupación del padre.
* reason: categórica. Es la razón para escoger escuela.
* guardian: categórica. Define el apoderado.
* traveltime: categórica. Especifica el tiempo de viaje entre el hogal y la escuela. Presenta rangos.
* studytime: categórica. Detalla las horas semanales de estudio.
* failures: numérica. Especifica el número de clases reprobadas.
* schoolsup: binaria. Si recibe o no apoyo educacional del colegio.
* famsup: binaria. Si recibe apoyo educacional familiar.
* paid. binaria. Si recibe clases particulares pagadas o no.
* activities: binaria. Si realiza actividades extracurriculares o no.
* nursery: binaria. Define si asistió o no a la guardería infantil.
* higher: binaria. Si desea proseguir estudios superiores.
* internet: binaria. Si tienes acceso a internet desde el hogar.
* romantic: binaria. Si tiene una relación amorosa.
* famrel: categórica ordinal. Define la calidad de las relaciones familiares.
* freetime: categórica ordinal. Tiempo libre fuera del colegio.
* goout: categórical ordinal. Define si sale con amigos.
* Dalc: categórica ordinal. Consumo de alcohol en días de semana.
* Walc: categórica ordinal. Consumo de alcohol en fines de semana.
* health: categórica ordinal. Estado de salud actual.
* absences: numérica. Cantidad de ausencias escolares.

En cuanto a las variables objetivo:

* G1: numérica. Notas durante el primer semestre (V.O 1 modelo descriptivo).
* G2: numérica. Notas durante el segundo semestre (V.O 2 modelo descriptivo).
* G3: numérica. Promedio final (V.O 3 modelo descriptivo, y V.O modelo predictivo).

#### 1.3 Métricas para medir desempeño

Para medir el desempeño de nuestro modelo utilizaremos dos medidas:

* Error cuadrático medio (MSE): representa la expectativa del error cuadrático, donde este se define a partir de las distancias entre el valor estimado y el valor real de la variable objetivo. Nos permite medir la calidad de nuestro ajuste, e idealmente queremos que sea pequeño.

* r2_score: representa la proporción porcentual de la varianza total de la variable objetivo que es explicada por los atributos agregados al modelo.

### 2. Aspectos computacionales

#### 2.1 Librerias a utilizar

Las librerias y versiones utilizadas son:

* numpy 
* pandas
* matplotlib
* seaborn
* warnings
* sklearn

Además, trabajaremos con la versión de Python 3.8.3.

#### 2.2 Proceso tentativo recodificación de atributos

Se hará un análisis exploratorio tanto de la variable objetivo como de sus atributos donde:

* Se identificará la presencia de valores nulos, perdidos o caracteres incorrectos, y serán eliminados del modelo ya que nos generan problemas en la estimación. En esta etapa, se utilizará la función importada "calidad_datos" que nos permite tener una perspectiva global.

* Estudio de dimensionalidad. Se aplicará el algoritmo de Kmeans de la librería Sklearn para estudiar la medida de similitud entre los atributos y considerar agrupar variables.

* Transformación de variables categóricas mediante el uso de binary encoding.

* Estudio de la escala es la cual están definidas las variables numéricas. Si existen diferencias muy grandes en la dimensionalidad de los números se aplicará una estandarización con StandardScaler() de Sklearn.

* Transformación de variables binarias a 1 y 0. Se considerará como 1 la categoría minoritaria.

* Exploración de correlaciones entre atributos y variable objetivo. La idea es identificar atributos que se correlacionan fuertemente y débilmente, para considerar excluirlas del modelo.

#### 2.3 Importación de librerias

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# machine learning libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from calidad_datos import calidad_datos

plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["figure.dpi"] = 70
warnings.filterwarnings('ignore')

### 3. Descripción

In [6]:
# importación de datos
df = pd.read_csv('students.csv', sep='|', encoding='latin-1').drop('Unnamed: 0', axis=1)

In [7]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,nulidade,U,GT3,A,4,4,at_home,teacher,...,4,3,"""4""",1,1,"""3""",6,5,6,6
1,GP,F,"""17""",U,GT3,T,1,1,at_home,other,...,5,3,"""3""",1,1,"""3""",4,5,5,6
2,GP,F,"""15""",U,LE3,T,1,1,at_home,other,...,4,3,"""2""",2,3,"""3""",10,zero,8,10
3,GP,F,"""15""",U,GT3,T,4,2,health,services,...,3,2,"""2""",1,1,"""5""",2,15,14,15
4,GP,F,sem validade,U,GT3,T,3,3,other,other,...,4,3,"""2""",1,2,"""5""",4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,"""20""",U,LE3,A,2,2,services,services,...,5,5,"""4""",4,5,"""4""",11,9,9,9
391,MS,M,"""17""",U,LE3,T,3,1,services,services,...,2,4,"""5""",3,4,"""2""",3,14,16,16
392,MS,M,"""21""",R,GT3,T,1,1,other,other,...,5,5,"""3""",3,3,"""3""",3,10,8,7
393,MS,M,"""18""",R,LE3,T,3,2,services,other,...,4,4,"""1""",3,4,"""5""",0,11,12,10


In [ ]:
#df_s.age.str.replace('"', "").replace(['nulidade','sem validade'], [np.nan, np.nan]).astype('float')